# Learning how to simulate urdfs:

In [1]:
import pybullet as p
import time
import numpy as np
import matplotlib.pyplot as plt
import pybullet_data

pybullet build time: May 20 2022 19:44:17


In [2]:
physicsClient = p.connect(p.GUI)#or p.DIRECT for non-graphical version
p.setAdditionalSearchPath(pybullet_data.getDataPath()) #this will load the plane urdf 
p.setGravity(0,0,-10) #along the Z axis
planeId = p.loadURDF("plane.urdf")

#---loading the robotID----:
startPos = [0,0,1] # it will be spawned at z=1
startOrientation = p.getQuaternionFromEuler([0,np.pi/2,0]) #angle at which it will be spawned
#robot = p.loadURDF("/home/anushtup/Documents/Coding/Robot_sims/sim_env/Jumping/jumping.urdf",startPos, startOrientation)
robot = p.loadURDF("/home/anushtup/Documents/Coding/Robot_sims/sim_env/Jumping/jumping.urdf",startPos, startOrientation, useFixedBase = False)

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=AMD
GL_RENDERER=AMD RENOIR (DRM 3.42.0, 5.15.0-60-generic, LLVM 12.0.0)
GL_VERSION=4.6 (Core Profile) Mesa 21.2.6
GL_SHADING_LANGUAGE_VERSION=4.60
pthread_getconcurrency()=0
Version = 4.6 (Core Profile) Mesa 21.2.6
Vendor = AMD
Renderer = AMD RENOIR (DRM 3.42.0, 5.15.0-60-generic, LLVM 12.0.0)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = AMD
ven = AMD


In [3]:
p.setGravity(0,0,-10)
p.setRealTimeSimulation(1)

In [4]:
for i in range(p.getNumJoints(robot)):
    print(p.getJointInfo(robot,i))

(0, b'hfe_joint', 0, 7, 6, 1, 0.0, 0.0, -1.6, 1.6, 200.0, 10.0, b'upperleg', (0.0, 0.0, 1.0), (0.2, 0.0, 0.0), (-0.7071067966408575, 0.0, 0.0, 0.7071067657322372), -1)
(1, b'kfe_joint', 0, 8, 7, 1, 0.0, 0.0, -1.6, 1.6, 200.0, 10.0, b'lowerleg', (0.0, 0.0, 1.0), (0.35, 0.0, 0.0), (0.0, 0.0, 0.0, 1.0), 0)
(2, b'lowerleg_contactsensor_link_joint', 4, -1, -1, 0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, b'lowerleg_contactsensor_link', (0.0, 0.0, 0.0), (0.35, 0.0, 0.0), (0.0, 0.0, 0.0, 1.0), 1)


In [6]:
jointNameToID = {}
linkNameToID = {}
revoluteID = []


for j in range(p.getNumJoints(robot)):
    info = p.getJointInfo(robot, j)
    # print(info)
    jointID = info[0]
    jointName = info[1].decode('UTF-8')
    jointType = info[2]
    if (jointType == p.JOINT_REVOLUTE):
        jointNameToID[jointName] = info[0]
        linkNameToID[info[12].decode('UTF-8')] = info[0]
        revoluteID.append(j)
print(linkNameToID)

{'upperleg': 0, 'lowerleg': 1}


In [33]:
jointID = 1
jLow1 = p.getJointInfo(robot, jointID)[8] 
jUp1 = p.getJointInfo(robot, jointID)[9]
jytpe1 = p.getJointInfo(robot, jointID)[2]

print(jLow1, jUp1)
print(p.getLinkStates(robot, [2])[0][1])

-1.6 1.6
(0.9943264939880689, 0.08876735068432552, 0.011286206367809668, 0.057513497126361855)


In [8]:
for step in range( 500 ):
    joint_0_targ = np.random.uniform(jLow1, jUp1)
    joint_1_targ = np.random.uniform(jLow1, jUp1)

    p.setJointMotorControlArray(robot, [0,1], p.POSITION_CONTROL, targetPositions = [joint_0_targ, joint_1_targ])
    p.stepSimulation()
    
    time.sleep(0.01)
    # print(p.getLinkStates(robot, [1]))



In [9]:
pivot=[0,0,0,1]
decomprPhase=0
xbody,orQT = p.getBasePositionAndOrientation(robot)
orDes=orQT
orEU=p.getEulerFromQuaternion(orQT)
t=0
tstr=0


In [35]:
for i in range(1000):
    t+=1
    time.sleep(.01)
    xbodyprv=xbody
    orEUprv=orEU
    xbody,orQT = p.getBasePositionAndOrientation(robot)
    vxbody=(xbody[0]-xbodyprv[0])/0.01
    vybody=(xbody[1]-xbodyprv[1])/0.01
    vzbody=(xbody[2]-xbodyprv[2])/0.01
    orEU=p.getEulerFromQuaternion(orQT)
    omgx=(orEU[0]-orEUprv[0])/0.01
    omgy=(orEU[1]-orEUprv[1])/0.01

    xfoot,dum = p.getLinkStates(robot, [2])[0][0],p.getLinkStates(robot, [2])[0][1]

    if vzbody>0 and decomprPhase==0:
        decomprPhase=1
        tstr=t
    #lift off condition: foot is off the ground
    if xfoot[2]>0.105 and decomprPhase==1:
        decomprPhase=0

    if decomprPhase==1:
        #decompressing: PD control on orientation of body during stance
        #orDesEU=p.getEulerFromQuaternion(orDes)
        #orDes = p.getQuaternionFromEuler(orDesEU + np.array([-0.07*omgx-0.3*orEU[0],
        #                                                     -0.07*omgy-0.3*orEU[1],0.0]))
        if ((t-tstr)<8):  #trust for a small time interval (increased spring force)
            p.changeConstraint(cid,pivot,jointChildFrameOrientation=orDes, maxForce=60)
        else:
            p.changeConstraint(cid,pivot,jointChildFrameOrientation=orDes, maxForce=30)
    else:
        #flight and compression: Reposition foot for next landing based on body horizontal velocity and orientation
        if (xfoot[2]>0.105):
            orDes = p.getQuaternionFromEuler(
                    [+0.2*(-(vybody-0.0)*np.cos(orEU[2])+(vxbody-0.0)*np.sin(orEU[2])) + orEU[0],
                     +0.2*( (vxbody-0.0)*np.cos(orEU[2])+(vybody-0.0)*np.sin(orEU[2])) + orEU[1], 0.0])
        p.changeConstraint(cid,pivot,jointChildFrameOrientation=orDes, maxForce=30)
        destination.append(orDes)

p.removeConstraint(cid)
p.disconnect()

NameError: name 'cid' is not defined